# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [20]:
import sqlite3    # Python interface for SQLite databases

In [21]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [106]:
conn = sqlite3.connect('C://Users//Max Tchibozo//Desktop//wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER1sub1:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
MASTER1sub2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
MASTER1sub3:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
  3: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
MASTER2sub1:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
MASTER2sub2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
MASTER2sub3:
  0: LIEU(TEXT)
  1: REGION(TEXT)
MASTER2sub4:
  0: NB(NUM)
  1: NV(NUM)
  2: LIEU(TEXT)
  3: QTE(NUM)
  4: DATES(NUM)


From this point we will use __%%sql__ magic

In [23]:
%load_ext sql
%sql sqlite:///wine.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @wine.db'

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [24]:
%sql SELECT * FROM Master2 ORDER BY lieu;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,DATES,LIEU,QTE,NB,NOM,PRENOM,TYPE,REGION
None,None,None,None,None,None,None,11,Breton,Andre,petit,None
None,None,None,None,None,None,None,13,Barthes,Roland,moyen,None
None,None,None,None,None,None,None,16,Balzac,Honore de,moyen,None
None,None,None,None,None,None,None,18,Celine,Louis Ferdinand,gros,None
None,None,None,None,None,None,None,20,Chateaubriand,Francois-Rene de,moyen,None
None,None,None,None,None,None,None,21,Corbiere,Tristan,petit,None
None,None,None,None,None,None,None,23,Corneille,Pierre,petit,None
None,None,None,None,None,None,None,25,Char,Rene,petit,None
None,None,None,None,None,None,None,27,Dumas,Alexandre,gros,None
None,None,None,None,None,None,None,29,Fournier,Alain,petit,None


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [25]:
%%sql SELECT *
FROM Master1
WHERE nv=45;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
45,Chiroubles,None,1983,90,2,Boxler,Albert,Alsace
45,Chiroubles,None,1983,912,67,Descombes,Jean Ernest,Beaujolais
45,Chiroubles,None,1983,98,71,Chalandard,Danile,Jura
45,Chiroubles,None,1983,540,78,Michlel,Pierre Emile,Jura
45,Chiroubles,None,1983,450,86,Dumazet,Marc,Rhone


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [26]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

 * sqlite:///wine.db
Done.
Done.


[]

In [27]:
print("\nContent of the database")
printSchema(conn)


Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER1sub1:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
MASTER1sub2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
MASTER1sub3:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
MASTER2sub1:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
MASTER2sub2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
MASTER2sub3:
  0: LIEU(TEXT)
  1: REGION(TEXT)
MASTER2sub4:
  0: NB(NUM)
  1: NV(NUM)
  2: LIEU(TEXT)
  3: QTE(NUM)
  4: DATES(NUM)
dummy:
  0: DEGRE(NUM)


In [28]:
%%sql
SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [14]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.
Done.
Done.


DEGRE
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8
10.7


Notice that we use `NULL` given that `None` is only used for display.

#### Exercise 1.3

Create the new tables from Master1:

In [80]:
%%sql 

-- Create first subtable of master 1
CREATE TABLE MASTER1sub1 AS
SELECT DISTINCT NP,NOM,PRENOM
FROM MASTER1
WHERE NP IS NOT NULL;

-- Create second subtable of master 1
CREATE TABLE MASTER1sub2 AS
SELECT DISTINCT NV,CRU,DEGRE,MILL
FROM MASTER1
WHERE NV IS NOT NULL;


-- Create third subtable of master 1
CREATE TABLE MASTER1sub3 AS
SELECT DISTINCT NP,NV,QTE,REGION
FROM MASTER1
WHERE NP IS NOT NULL AND NV IS NOT NULL;



 * sqlite:///wine.db
Done.
Done.
Done.


[]

#### Exercise 1.4

Create the new tables from Master2:

In [34]:
%%sql 

-- Create first subtable of master 2
CREATE TABLE MASTER2sub1 AS
SELECT DISTINCT NB,NOM,PRENOM,TYPE
FROM MASTER2
WHERE NB IS NOT NULL;

-- Create second subtable of master 2
CREATE TABLE MASTER2sub2 AS
SELECT DISTINCT NV,CRU,DEGRE,MILL
FROM MASTER2
WHERE NV IS NOT NULL;


-- Create third subtable of master 2
CREATE TABLE MASTER2sub3 AS
SELECT DISTINCT LIEU,REGION
FROM MASTER2
WHERE LIEU IS NOT NULL;

-- Create fourth subtable of master 2
CREATE TABLE MASTER2sub4 AS
SELECT DISTINCT NB,NV,LIEU,QTE,DATES
FROM MASTER2
WHERE NB IS NOT NULL AND NV IS NOT NULL AND LIEU IS NOT NULL;


 * sqlite:///wine.db
(sqlite3.OperationalError) table MASTER2sub1 already exists [SQL: '-- Create first subtable of master 2\nCREATE TABLE MASTER2sub1 AS\nSELECT DISTINCT NB,NOM,PRENOM,TYPE\nFROM MASTER2\nWHERE NB IS NOT NULL;'] (Background on this error at: http://sqlalche.me/e/e3q8)


# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [30]:
%%sql 
SELECT DISTINCT TYPE
FROM MASTER2sub1;

 * sqlite:///wine.db
Done.


TYPE
petit
moyen
gros


#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [44]:
%%sql 

SELECT REGION
FROM MASTER1sub2 NATURAL JOIN MASTER1sub3
WHERE CRU = 'Pommard' OR CRU = 'Brouilly';

 * sqlite:///wine.db
Done.


CRU,REGION
Pommard,Bourgogne
Pommard,Rhone
Brouilly,Bourgogne


#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [50]:
%%sql 

SELECT REGION
FROM MASTER1sub2 NATURAL JOIN MASTER1sub3
WHERE CRU = 'Pommard' INTERSECT 
SELECT REGION
FROM MASTER1sub2 NATURAL JOIN MASTER1sub3
WHERE CRU = 'Brouilly';

 * sqlite:///wine.db
Done.


REGION
Bourgogne


#### Exercise 2.4

Get the number of wines bought by CRU and Millésime

In [54]:
%%sql 

SELECT QTE AS nb_of_wines_bought,CRU, MILL
FROM MASTER2sub2
NATURAL JOIN MASTER2sub4
GROUP BY CRU,MILL;

 * sqlite:///wine.db
Done.


nb_of_wines_bought,CRU,MILL
8,Arbois,1980
80,Auxey Duresses,1914
4,Beaujolais Primeur,1983
10,Beaujolais Villages,1975
120,Beaujolais Villages,1976
130,Beaujolais Villages,1978
500,Beaujolais Villages,1979
20,Chapelle Chambertin,1973
4,Chateau Corton Grancey,1980
1,Chenas,1984


#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [74]:
%%sql 

SELECT NV, COUNT(DISTINCT(NP)) AS number_of_producers
FROM MASTER1sub3 NATURAL JOIN MASTER1sub1 NATURAL JOIN MASTER1sub2
GROUP BY NV
HAVING COUNT(DISTINCT(NP))>3
;

 * sqlite:///wine.db
Done.


NV,COUNT(DISTINCT(NP))
45,5
78,5
89,4
98,5


#### Exercise 2.6

Which producers have not produced any wine?

In [136]:
%%sql 
SELECT NP,NOM,PRENOM FROM MASTER1sub1
WHERE NP IN (
SELECT NP
FROM MASTER1sub1
EXCEPT 
SELECT NP
FROM MASTER1sub3)
;

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM
3,Six,Paul
6,Marmagne,Bernard
8,Lioger d'Harduy,Gabriel
16,Barbin,Bernard
17,Faiveley,Guy
18,Tramier,Jean
19,Dupaquier,Roger
20,Lamy,Jean
21,Cornu,Edmond
26,Violot,Gilbert


#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [120]:
%%sql 
SELECT NB,NOM,PRENOM
FROM MASTER2sub1
NATURAL JOIN MASTER2sub4
NATURAL JOIN MASTER2sub2
WHERE MILL = 1980
GROUP BY NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
2,Artaud,Antonin
8,Aragon,Louis
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None
61,Mallarme,Stephane


#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [128]:
%%sql 
SELECT NB,NOM,PRENOM FROM MASTER2sub1
WHERE NB IN (
SELECT NB
FROM MASTER2sub1
EXCEPT
SELECT NB
FROM MASTER2sub4
NATURAL JOIN MASTER2sub2
WHERE MILL =1980)
ORDER BY NB
;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
1,Aristote,None
3,Aron,Raymond
4,Apollinaire,Guillaume
5,Audiberti,Jacques
6,Arrabal,Fernando
7,Anouilh,Jean
9,Ajar,Emile
10,Andersen,Yann
11,Breton,Andre
12,Bataille,Georges


#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [194]:
%%sql 
SELECT NB, NOM, PRENOM
FROM MASTER2sub1 NATURAL JOIN MASTER2sub4 NATURAL JOIN MASTER2sub2
EXCEPT
SELECT NB, NOM, PRENOM
FROM MASTER2sub1 NATURAL JOIN MASTER2sub4 NATURAL JOIN MASTER2sub2
WHERE MILL != 1980

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None


#### Exercise 2.10

List all wines from 1980

In [193]:
%%sql 
SELECT  DISTINCT CRU
FROM MASTER2sub2
NATURAL JOIN MASTER2sub1
WHERE MILL = 1980

 * sqlite:///wine.db
Done.


CRU
Arbois
Chateau Corton Grancey
Cornas
Cote de Brouilly
Cotes de Provence
Etoile
Fleurie
Mercurey
Meursault
Rasteau


#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [190]:
%%sql 

SELECT NV,CRU,MILL,DEGRE
FROM MASTER2sub2
NATURAL JOIN MASTER2sub4
WHERE MILL = 1980 AND NB=2;

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5


#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [259]:
%%sql
SELECT NB, NOM, PRENOM
FROM(SELECT MAX(count_nv), NB
FROM(
SELECT COUNT(NV) AS count_nv,NB
FROM MASTER1sub1
NATURAL JOIN MASTER2sub4
NATURAL JOIN MASTER2sub2
WHERE MILL = 1980
GROUP BY NB))
NATURAL JOIN MASTER2sub4
NATURAL JOIN MASTER2sub1
GROUP BY NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
